In [121]:
import pandas as pd
import os
import plotly
import plotly.express as px
import plotly.graph_objects as go
#import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import numpy as np 
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

#app = dash.Dash(__name__ , external_stylesheets= [dbc.themes.CYBORG])
app = JupyterDash(__name__, external_stylesheets= [dbc.themes.CYBORG])

In [122]:
def make_total_dataframe():
    """ Making a global stat_conf dataframe """
    all_stat = []
    conf = [f for f in os.listdir('data') if '_meta' in f ]
    for c in conf:
        df = pd.read_csv('data/' + c, sep = '\t')

        df = df[['primary_id', 'latitude', 'longitude', 'station_name']]
        inv = c.split('_meta')[0]
        df['dataset'] = inv

        all_stat.append(df)

    all_df = pd.concat(all_stat)

    return all_df


all_stat_conf = make_total_dataframe()

In [123]:
""" Setting the APP LAYOUT 
What goes inside the app layout is your dash components,
with the graphs, layouts, checkboxes
anything that is listed here: https://dash.plotly.com/dash-core-components 

"""

app.layout = html.Div([
    # title of our web page
    html.H1("Distributions of Stations", style={'text-align': 'center'}),
    html.Br(),  # Br is a break i.e. a space in between
    # label and the key is what actually the user is going to see
    # values are integer since they come from the pandas dataframe and they are integer there

    dcc.Dropdown(id="dataset",

                 options=[
                     {"label": "ERA5_1", "value": "1"},
                     {"label": "ERA5_2", "value": "2"},
                     {"label": "ERA5_1759", "value": "1759"},
                     {"label": "ERA5_1761", "value": "1761"},
                     {"label": "ERA5_3188", "value": "3188"},
                     {"label": "NCAR", "value": "rda"},
                     {"label": "IGRA2", "value": "igra2"},
                 ],

                 multi=False,
                 value="1", # this is the initial value displayed in the dropwdow
                 style={'width': "40%"}
                 ),

    #html.Div(id='output_container', children=[]),
    html.Br(),  # Br is a break i.e. a space in between
    dcc.Graph(id='mappa', figure={} , style={'width': '100vh', 'height': '60vh'}),
    html.Br(),  # Br is a break i.e. a space in between
    dcc.Graph(id='bars', figure={}, style={'width': '90vh', 'height': '50vh'})
    
    #dcc.Graph(id='mappa', figure={}),
    #html.Br(),  # Br is a break i.e. a space in between
    #dcc.Graph(id='bars', figure={})
    

])

In [124]:
@app.callback(
    [Output(component_id='mappa', component_property='figure'),
     Output(component_id='bars', component_property='figure')
     ],
    [Input(component_id='dataset', component_property='value')])

def update_plots(dataset):

    """ map plot """
    font = 15
    # select the dataset
    df = all_stat_conf.loc [ all_stat_conf['dataset'] == dataset ]


        
    def do_map(df):
        miss, valid = [],[]
        ids = df['primary_id']
        
        for i,ind in zip(ids,range(len(ids)) ):
            if '0-20' not in i:
                    miss.append(ind)
            else:
                valid.append(ind)
                
        miss_df = df.iloc[miss]
        df_valid = df.loc[valid]
        fig = go.Figure()
        
        fig.add_trace( go.Scattergeo(lat=df_valid.latitude, 
                                     lon=df_valid.longitude,
                                     text = df_valid.primary_id,
                                     name = 'Identified  ' + str(len(df_valid))
                             #hover_name="primary_id",
                             #hover_data = ['station_name'],
                             #width=1000, height=600
                             ) )
        
        fig.add_trace(go.Scattergeo (lat=miss_df.latitude, 
                                     lon=miss_df.longitude,
                                     name = 'Missing  ' + str(len(miss_df))
                             #hover_name="primary_id",
                             #hover_data = ['station_name'],
                             #width=1000, height=600
                              ) )

        if dataset =='1759':   
            mismatch = pd.read_csv('data/lat_mismatch.csv', sep = '\t')        
            
            fig.add_trace(go.Scattergeo (lat=mismatch.file_lat, 
                                     lon=mismatch.file_lon,
                                     text = df.primary_id,
                                     name = 'Latitude mismatch  ' + str(len(mismatch))
                             #hover_name="primary_id",
                             #hover_data = ['station_name'],
                             #width=1000, height=600
                              ) )
        
        
        fig.update_layout(
            title_text= '[' + str(len(df)) + ']  Station (1900-2020)' ,
            # title_xanchor="center",
            title_font=dict(size=font + 6), title_x=0.5,
            font=dict( size=font, ),
            hoverlabel=dict( font_size=18, )
        )

        return fig

    def do_bars(df):
        inventories = ['oscar', 'igra2', 'wban', 'chuan', 'missing']

        counts = {}
        
        primary = df['primary_id']
        for l in ['20000-', '20001-', '20300-', '20400-', '20500-']:
            counts[l] = len([ s for s in primary if l in s ])


        data = [ counts['20000-'] + counts['20001-'],
                counts['20300-'],
                counts['20400-'],
                counts['20500-'] ]

        counts['missing'] = len(df) - sum(data)
        
        data.append(counts['missing'])

        bar = px.bar(data, x=inventories, y=data, title="Inventories" )

        return bar

    mappa = do_map(df)
    bars = do_bars(df)

    return [mappa, bars] # NB must always return a list even if you have one output only, due to @app


In [125]:
if __name__ == '__main__':
    #app.run_server(mode = 'inline', debug=True)
    app.run_server(mode='inline')
    # app.run_server(host='0.0.0.0', debug=True)